In [1]:
import json
from jsonpath import jsonpath
import pandas as pd
import re
import os
import glob
import pdfkit
import imgkit
import urllib.request
from bs4 import BeautifulSoup

In [2]:
# 遍历文件夹下的 JSON 文件，循环读取内容
folder_path = './我的公众号JSON'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_form = '*.json'
files_name = os.path.join(folder_path,file_form)
all_files = glob.glob(files_name)
all_files

['./我的公众号JSON\\106_.json',
 './我的公众号JSON\\117_.json',
 './我的公众号JSON\\12_.json',
 './我的公众号JSON\\130_.json',
 './我的公众号JSON\\142_.json',
 './我的公众号JSON\\153_.json',
 './我的公众号JSON\\165_.json',
 './我的公众号JSON\\179_.json',
 './我的公众号JSON\\190_.json',
 './我的公众号JSON\\1_.json',
 './我的公众号JSON\\201_.json',
 './我的公众号JSON\\213_.json',
 './我的公众号JSON\\224_.json',
 './我的公众号JSON\\235_.json',
 './我的公众号JSON\\246_.json',
 './我的公众号JSON\\257_.json',
 './我的公众号JSON\\268_.json',
 './我的公众号JSON\\26_.json',
 './我的公众号JSON\\280_.json',
 './我的公众号JSON\\291_.json',
 './我的公众号JSON\\302_.json',
 './我的公众号JSON\\313_.json',
 './我的公众号JSON\\324_.json',
 './我的公众号JSON\\335_.json',
 './我的公众号JSON\\33_.json',
 './我的公众号JSON\\346_.json',
 './我的公众号JSON\\357_.json',
 './我的公众号JSON\\35_.json',
 './我的公众号JSON\\369_.json',
 './我的公众号JSON\\379_.json',
 './我的公众号JSON\\390_.json',
 './我的公众号JSON\\400_.json',
 './我的公众号JSON\\413_.json',
 './我的公众号JSON\\424_.json',
 './我的公众号JSON\\435_.json',
 './我的公众号JSON\\446_.json',
 './我的公众号JSON\\456_.json',
 './我的公

In [10]:
# 获取单个 JSON 文件中的 文章标题、网页链接  
df = pd.DataFrame(columns=['文章标题','网页链接'])

for file in all_files:

    title_list = []
    content_url_list = []

    try:
        json_data = json.load(open(file,'r',encoding='utf-8'))
        data_list = json.loads(json_data['general_msg_list'])['list']
        for data in data_list:           
            title = data['app_msg_ext_info']['title']
            content_url = data['app_msg_ext_info']['content_url']                   
            title_list.append(title)
            content_url_list.append(content_url)
    except:
        pass

    df_json = pd.DataFrame({'文章标题':title_list,'网页链接':content_url_list})
    df = pd.concat([df,df_json],ignore_index=True)

df['文章标题'] = df['文章标题'].str.replace('|','-')
df['文章标题'] = df['文章标题'].str.replace('/','-')
df['文章标题'] = df['文章标题'].str.replace('\n','')
df['文章标题'] = df['文章标题'].str.replace('～','')
df['文章标题'] = df['文章标题'].str.replace('*','')
df['文章标题'] = df['文章标题'].str.replace('?','')
df = df[df.文章标题.str.len() < 40]
df = df[df.网页链接 != '']

df

D:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
D:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
D:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.


,文章标题,网页链接
1,一日一技：如何为 Python 基本类型自定义方法？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
2,如何改造 Scrapy 从而实现多网站大规模爬取？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
3,一日一技：在 Golang 中如何快速判断字符串是否在一个数组中,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
4,Python 为什么不支持 switch 语句？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
5,为什么别人不想回答你提的问题？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
...,...,...
530,Python 之父为什么嫌弃 lambda 匿名函数？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
531,一日一技：除了 time.sleep，你还有一个暂停代码的方法,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
532,一日一技：如何从 URL 中快速提取域名？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...
533,为什么说 Python 内置函数并不是万能的？,http://mp.weixin.qq.com/s?__biz=MzI2MzEwNTY3OQ...


In [11]:
# 循环读取 df 中的网址，保存为本地网页
def getHtml(url):
    html = urllib.request.urlopen(url).read()
    return html
 
def saveHtml(file_name, file_content):        
    with open(file_name + ".html", "wb") as f:
        f.write(file_content)
        
folder_name = './我的公众号网页'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
for url,title in zip(df['网页链接'],df['文章标题']):
    file_name = os.path.join(folder_name,title)
    html = getHtml(url)
    saveHtml(file_name,html)
    print(f'{title} 网页保存成功！！')
print('所有网页保存完毕！！')

一日一技：如何为 Python 基本类型自定义方法？ 网页保存成功！！
如何改造 Scrapy 从而实现多网站大规模爬取？ 网页保存成功！！
一日一技：在 Golang 中如何快速判断字符串是否在一个数组中 网页保存成功！！
Python 为什么不支持 switch 语句？ 网页保存成功！！
为什么别人不想回答你提的问题？ 网页保存成功！！
一日一技：如何防止 Homebrew 自动升级和自动清理过期程序 网页保存成功！！
Python 爬虫进阶必备 - 某指数分析平台解密逻辑分析 网页保存成功！！
一日一技：在网页上如何获取鼠标当前指向的元素 网页保存成功！！
奇技淫巧：在 ssh 里面把服务器的文本复制到本地电脑 网页保存成功！！
一日一技：在Golang下如何相对简单地开发爬虫？ 网页保存成功！！
一日一技：loguru 如何把不同的日志写入不同的文件中 网页保存成功！！
~与产品经理度过的第二个生日~ 网页保存成功！！
一日一技：导入父文件夹中的模块并读取当前文件夹内的资源 网页保存成功！！
2020年非科班转行数据分析可行吗？ 网页保存成功！！
一起读源码：为什么 loguru 的时间 rotation 不能只精确到天 网页保存成功！！
今天在收拾行李准备离开杭州。休更三天。周日恢复。 网页保存成功！！
手把手教学，如何解决 git 冲突？ 网页保存成功！！
什么叫做类比，为什么有些 Python 入门教程结构不合理？ 网页保存成功！！
一日一技：FastAPI如何关闭接口文档？ 网页保存成功！！
现场打脸：如何使用Selenium批量上传文件？ 网页保存成功！！
在 Python 中如何快速创建一个只读字典？ 网页保存成功！！
我这几年踩过的十个坑，每一条都是血泪教训 网页保存成功！！
Python不香吗？他竟然要转行…… 网页保存成功！！
来了来了评上了2020-2021年度微软MVP。据说全国只有150多人。 网页保存成功！！
爬虫工程师也应该会的 NodeJS 知识（三）- 快速抛弃 execjs 网页保存成功！！
用个小技巧，趁你不备，rm -rf你的电脑 网页保存成功！！
一日一技：pylint除了检查代码风格，还能做这件事情 网页保存成功！！
大家七夕快乐 网页保存成功！！
一日一技：用 Python 如何正确读取资源文件 网页保存成功！！


In [12]:
#读取文件
def ReadFile(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        all_the_text = f.read()
    return all_the_text

#定义一个保存文件的函数
def SaveFile(fpath,fileContent):
    with open(fpath, 'w', encoding='utf-8') as f:
            f.write(fileContent)

#修改网页中图片的src，使图片能正常显示
def ChangeImgSrc(htmlsource):
    bs = BeautifulSoup(htmlsource,"lxml") #由网页源代码生成BeautifulSoup对象，第二个参数固定为lxml
    imgList = bs.findAll("img") #找出网页中所有的<img 标签
    for img in imgList:
        originalURL = ""  # 定义一个变量保存图片真实url
        if "data-src" in img.attrs: #防止有的<img 标签中可能没有data-src而出错
            originalURL = img.attrs['data-src']
        elif "src" in img.attrs: #如果有src则提取出来
            originalURL = img.attrs['src']
        else:
            originalURL = ""
        if originalURL.startswith("//"):#如果url以//开始，则需要添加http：
            originalURL = "http:" + originalURL
        img.attrs["src"] = originalURL
    return str(bs) #将BeautifulSoup对象再置换为字符串，用于保存


folder_path = './我的公众号网页'
file_form = '*.html'
files_name = os.path.join(folder_path,file_form)
all_files = glob.glob(files_name)

    
for file in all_files:
    htmlstr = ReadFile(file)
    htmlstr = ChangeImgSrc(htmlstr)
    SaveFile(file,htmlstr)
    print(f'{file} 网页替换完成！！')
print('所有网页替换完成！！')

./我的公众号网页\10月22日晚来 B 站看我直播.html 网页替换完成！！
./我的公众号网页\2018年总结，2019年计划.html 网页替换完成！！
./我的公众号网页\2020年最全的人工智能学习路径！文末领取福利.html 网页替换完成！！
./我的公众号网页\2020年非科班转行数据分析可行吗？.html 网页替换完成！！
./我的公众号网页\2021元旦快乐.html 网页替换完成！！
./我的公众号网页\4种方法解决MongoDB游标超时的问题.html 网页替换完成！！
./我的公众号网页\a is b 为 True，a == b 一定为 True 吗？.html 网页替换完成！！
./我的公众号网页\alias导致virtualenv异常的分析和解法.html 网页替换完成！！
./我的公众号网页\APP爬虫-frida-某资讯app逆向过程.html 网页替换完成！！
./我的公众号网页\Bug or Feature？藏在 requests_html 中的陷阱.html 网页替换完成！！
./我的公众号网页\Bytes 陷阱， Redis 数据类型的一个小坑.html 网页替换完成！！
./我的公众号网页\Bytes型数据decode时是如何知道要把几位数据组合在一起的？.html 网页替换完成！！
./我的公众号网页\Callback ——从同步思维切换到异步思维.html 网页替换完成！！
./我的公众号网页\Chome 88如何正确隐藏 webdriver？.html 网页替换完成！！
./我的公众号网页\Gne Online：通用新闻网页正文在线提取.html 网页替换完成！！
./我的公众号网页\GNE v0.04版更新，支持提取正文图片与源代码.html 网页替换完成！！
./我的公众号网页\GNE v0.1正式发布：4行代码开发新闻网站通用爬虫.html 网页替换完成！！
./我的公众号网页\JavaScript 混淆与逆向必读之 AST 节点类型名词基础.html 网页替换完成！！
./我的公众号网页\JavaScript 逆向爬取实战.html 网页替换完成！！
./我的公众号网页\JavaScript 逆向爬取实战（下）.html 网页替换完成！！
./我的公众号网页\JS逆向时碰到了恶心的死代码怎么办？手把手教你解决

In [ ]:
# 循环读取 df 中的网址，对网页进行 PDF 转化
folder_name = './我的公众号PDF'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
config = pdfkit.configuration(wkhtmltopdf=r'D:\software\wkhtmltopdf\bin\wkhtmltopdf.exe')
wkhtmltopdf_options = {'enable-local-file-access': None,'quiet': ''}

for url,title in zip(df['网页链接'],df['文章标题']):
    format_name = title + '.pdf'
    file_name = os.path.join(folder_name,format_name)
    pdfkit.from_url(url,file_name,configuration=config)
print('生成 PDF 文件成功！！')

In [ ]:
# 循环读取 df 中的网址，对网页进行图片转化
folder_name = './我的公众号图片'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    
config = imgkit.config(wkhtmltoimage=r'D:\software\wkhtmltopdf\bin\wkhtmltoimage.exe')
# wkhtmltopdf_options = {'enable-local-file-access': None}
options = {"encoding": "UTF-8",'quiet': ''}

for url,title in zip(df['网页链接'],df['文章标题']):
    format_name = title + '.jpg'
    file_name = os.path.join(folder_name,format_name)
    imgkit.from_url(url,file_name,config=config,options=options)
print('生成图片成功！！')